In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:1]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# class GANModel(GAN):
#     def __init__(self, input_channels, input_width, pretrained_model_path=None, **kwargs):
#         super().__init__(input_channels=input_channels, input_width=input_width, **kwargs)
#         self.input_width = input_width
#         self.save_hyperparameters()
#         self.pretrained_model_path = pretrained_model_path

#     def forward(self, z, target=None):
#         if target is not None:
#             Z = torch.sigmoid(z)
#             target = torch.sigmoid(target)
#             return F.binary_cross_entropy_with_logits(Z, target, weight=self.weight, reduction=self.reduction)
#         else:
#             return self.generator(z)

#     def configure_optimizers(self, lr = 0.0002, betas = (0.5, 0.999)):
#         generator_optimizer = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=betas)
#         discriminator_optimizer = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=betas)
#         return [generator_optimizer, discriminator_optimizer], []

#     def training_step(self, batch, batch_idx, optimizer_idx):
#         return super().training_step((batch, 0), batch_idx, optimizer_idx)

#     def validation_step(self, batch, batch_idx):
#         return super().validation_step((batch, 0), batch_idx)

#     def test_step(self, batch, batch_idx):
#         return super().test_step((batch, 0), batch_idx)

#     def prepare_data(self):
#         transform = transforms.Compose([
#             transforms.ToTensor(),
#             transforms.Resize((self.input_width, self.input_width)),
#         ])
#         self.train_dataset = CustomDataset(
#             root_dir='/kaggle/input/intel-image-classification/seg_train/seg_train/mountain/',
#             transform=transform,
#         )
#         self.val_dataset = CustomDataset(
#             root_dir='/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/',
#             transform=transform,
#         )
#         self.test_dataset = CustomDataset(
#             root_dir='/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/',
#             transform=transform,
#         )

#     def train_dataloader(self):
#         return DataLoader(self.train_dataset, batch_size=self.hparams['batch_size'])

#     def val_dataloader(self):
#         return DataLoader(self.val_dataset, batch_size=self.hparams['batch_size'])

#     def test_dataloader(self):
#         return DataLoader(self.test_dataset, batch_size=self.hparams['batch_size'])

#     def on_train_start(self):
#         if self.pretrained_model_path:
#             self.load_state_dict(torch.load(self.pretrained_model_path))

#     def generate_images(self, num_images):
#         z = torch.randn(num_images, self.latent_dim, device=self.device)
#         with torch.no_grad():
#             generated_images = self(z)
#         generated_images = generated_images.permute(0, 2, 3, 1).cpu().numpy()
#         generated_images = (generated_images * 255).astype(np.uint8)
#         return generated_images

#     def forward(self, input, target):
#         input = torch.sigmoid(input)
#         target = torch.sigmoid(target)
#         return F.binary_cross_entropy_with_logits(input, target, weight=self.weight, reduction=self.reduction)

In [ ]:
# !conda create --name env_name python=3.7

In [ ]:
!pip install -q torch==1.8.1
!pip install -q Pillow
!pip install -q lightning-lite==1.8.0
# !pip install -q pytorch-lightning==1.8.0
!pip install -q pytorch-lightning --upgrade
!pip install -q git+https://github.com/PytorchLightning/lightning-bolts.git@master --upgrade
!pip install -q lightning-utilities==0.3.0

In [ ]:
# !pip install -q torch==1.8.1
# !pip install -q Pillow
# !pip install -q lightning-lite==1.8.1
# !pip install -q pytorch-lightning==1.8.1
# !pip install -q lightning-bolts==0.6.0.post1
# !pip install -q lightning-utilities==0.3.0

In [ ]:
import os
import torch
import numpy as np
import pytorch_lightning as pl
from matplotlib import pyplot as plt
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils
from PIL import Image
from pl_bolts.models.gans import DCGAN, GAN, SRGAN, Pix2Pix
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities import rank_zero_only
import torch.nn.functional as F

from typing import Optional, Callable, Union, Any
from pytorch_lightning import LightningDataModule

from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        # get one sample from dataset
        image_name = os.path.join(self.root_dir, self.image_list[index])
        image = np.array(Image.open(image_name).convert('RGB'))  # convert to grayscale
        image = image.astype(np.float32)
        if self.transform:
            image = self.transform(image)
        return image, 0  # return the image as target

In [ ]:
class MyDataModule(pl.LightningDataModule):
    def __init__(self, transform, batch_size=16):
        super().__init__()
        self.batch_size = batch_size
        self.train_dataset = CustomDataset(
            root_dir='/kaggle/input/intel-image-classification/seg_train/seg_train/mountain/',
            transform=transform,
        )
        self.val_dataset = CustomDataset(
            root_dir='/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/',
            transform=transform,
        )
        self.test_dataset = CustomDataset(
            root_dir='/kaggle/input/intel-image-classification/seg_test/seg_test/mountain/',
            transform=transform,
        )

    def train_dataloader(self):
        # return your training dataloader
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        # return your validation dataloader
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        # return your test dataloader
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [ ]:
# class SRGAN(SRGAN):
#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         y_hat = self(x)
#         val_loss = F.mse_loss(y_hat, y)
#         return val_loss
    
#     def validation_step(self, batch, batch_idx):
#         x, y = batch
#         if y is None:  # skip if y is None
#             return
#         y_hat = x
#         val_loss = F.mse_loss(y_hat, y)
#         return val_loss
    
#     def forward(self, x):
#         print('Input shape:', x.shape)
#         x = self.input_block(x)

# class DCGAN(DCGAN):
#     def configure_optimizers(self):
#         if self.trainer.sanity_checking:
#             return torch.optim.Adam(self.parameters(), lr=1e-3)
#         elif self.trainer.current_epoch < 5:
#             return torch.optim.Adam(self.parameters(), lr=1e-3)
#         else:
#             return [torch.optim.SGD(self.parameters(), lr=1e-3, momentum=0.9),
#                     torch.optim.Adam(self.parameters(), lr=1e-4)]
    
#     def training_step(self, batch, batch_idx, optimizer_idx):
#         real_images, _ = batch
#         batch_size = real_images.size(0)

#         # Generate images from the noise
#         noise = torch.randn(batch_size, 100, 1, 1, device=self.device)
#         fake_images = self.generator(noise)

#         # Train the discriminator on the real images
#         if optimizer_idx == 0:
#             # Reset discriminator gradients
#             self.discriminator.zero_grad()

#             # Compute discriminator loss on real images
#             real_labels = torch.ones(batch_size, 1, device=self.device)
#             real_scores = self.discriminator(real_images)
#             d_real_loss = F.binary_cross_entropy_with_logits(real_scores, real_labels)

#             # Compute discriminator loss on fake images
#             fake_labels = torch.zeros(batch_size, 1, device=self.device)
#             fake_scores = self.discriminator(fake_images.detach())
#             d_fake_loss = F.binary_cross_entropy_with_logits(fake_scores, fake_labels)

#             # Compute total discriminator loss and update discriminator weights
#             d_loss = d_real_loss + d_fake_loss
#             self.manual_backward(d_loss)
#             self.discriminator_opt.step()

#             # Log discriminator loss
#             self.log("d_loss", d_loss, on_step=True, on_epoch=True, prog_bar=True)

#         # Train the generator
#         if optimizer_idx == 1:
#             # Reset generator gradients
#             self.generator.zero_grad()

#             # Generate fake images and compute generator loss
#             labels = torch.ones(batch_size, 1, device=self.device)
#             scores = self.discriminator(fake_images)
#             g_loss = F.binary_cross_entropy_with_logits(scores, labels)

#             # Update generator weights
#             self.manual_backward(g_loss)
#             self.generator_opt.step()

#             # Log generator loss
#             self.log("g_loss", g_loss, on_step=True, on_epoch=True, prog_bar=True)

#         return {'loss': d_loss + g_loss} # Return total loss for logging purposes

In [ ]:
#     checkpoint_callback = ModelCheckpoint(
#         dirpath='./models',
#         filename='vae-{epoch:02d}-{step:02d}-{val_loss:.2f}',
#         save_top_k=3,
#         verbose=True,
#         monitor='val_loss',
#         mode='min',
#         every_n_epochs=25  # set n to the desired number of iterations
#     )

#     # Run learning rate finder
#     lr_finder = trainer.tuner.lr_find(model, datamodule=data_module)

#     # Results can be found in
#     lr_finder.results

#     # Plot with
#     fig = lr_finder.plot(suggest=True)
#     fig.show()

#     # Pick point based on plot, or get suggestion
#     new_lr = lr_finder.suggestion()

#     # update hparams of the model
#     model.hparams.lr = new_lr

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam

class DCGAN(DCGAN):
    # example with learning rate schedulers
    def configure_optimizers(self):
        lr = self.hparams.learning_rate
        betas = (self.hparams.beta1, 0.999)
        opt_disc = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=betas)
        opt_gen = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=betas)
        disc_sch = ReduceLROnPlateau(opt_disc, mode='min', factor=0.1, patience=10, verbose=True)
        gen_sch = ReduceLROnPlateau(opt_gen, mode='min', factor=0.1, patience=10, verbose=True)
        return (
            {
                "optimizer": opt_disc,
                "lr_scheduler": {
                    "scheduler": disc_sch,
                    "monitor": "loss/disc",
                },
            }, {
                "optimizer": opt_gen, 
                "lr_scheduler": {
                    "scheduler": gen_sch,
                    "monitor": "loss/gen",
                },
            },
        )


In [ ]:
if __name__ == '__main__':
    pretrained_model_path = None
    batch_size = 4096

    gan_params = {
        'image_channels': 3, # input_channels
        'input_height': 64,
        'input_width': 64,
        'latent_dim': 512,
        'learning_rate': 0.000001,
        'feature_maps_gen': 64,
        'feature_maps_disc': 64,
    }

    # Initialize a trainer instance and train the model
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((gan_params['input_width'], gan_params['input_height']), antialias=True),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    model = DCGAN(**gan_params)

    checkpoint_path = 'checkpoint.pth'
    if os.path.exists(checkpoint_path):
        print(f"Loading from existing checkpoint: {checkpoint_path}")
        model.load_state_dict(torch.load(checkpoint_path))

    data_module = MyDataModule(transform, batch_size=batch_size)

#     trainer = pl.Trainer(
#         auto_lr_find=True,
#     )

#     trainer.tune(model, data_module)
#     # Find optimal learning rate
#     lr_finder = trainer.tune.lr_find(model, datamodule=data_module)
#     fig = lr_finder.plot(suggest=True)
#     plt.show()
    
#     # Set suggested learning rate
#     gan_params['learning_rate'] = lr_finder.suggestion()
    
#     # Learning rate logger
#     lr_logger = LearningRateLogger()
    
#     # Reduce learning rate on plateau
#     lr_scheduler = ReduceLROnPlateau(
#         monitor='val_loss', # the metric to monitor
#         factor=0.1,         # the factor to reduce the learning rate by
#         patience=10,        # the number of epochs with no improvement after which learning rate will be reduced
#         verbose=True,       # print a message when the learning rate is reduced
#         mode='min'          # the direction of the metric (minimizing val_loss)
#     )
    
    trainer = pl.Trainer(
#         callbacks=[lr_logger], # lr_scheduler
        accelerator='gpu',
        devices=1,
        sync_batchnorm=True,
        max_epochs=25,
        precision=32,
        auto_lr_find=True,
        val_check_interval=1.0,
        accumulate_grad_batches=1,
        gradient_clip_val=0.5,
        gradient_clip_algorithm="value",
        limit_train_batches=1.0,
        limit_val_batches=1.0,
        # log_every_n_steps=10,
        num_sanity_val_steps=1,
        # auto_scale_batch_size='power',  #auto_scale_batch_size='binsearch',
        # auto_select_gpus=True,
        overfit_batches=0.0,
        # plugins=None,
        # profiler=None,
        replace_sampler_ddp=True,
        # tpu_cores=None,
        track_grad_norm=-1,
        benchmark=True,
    )
    
    trainer.fit(
        model,
        datamodule=data_module
    )

    torch.save(model.state_dict(), checkpoint_path)

In [ ]:
# Generate and view images
model.eval()
with torch.no_grad():
    z = torch.randn(batch_size, model.hparams.latent_dim)
    generated_imgs = model(z)
    generated_imgs = generated_imgs.view(-1, 3, model.hparams.input_height, model.hparams.input_width)  # Reshape to (batch_size, channels, height, width)
    if batch_size >= 16:
        # Display up to 16 images in a 4x4 grid
        num_images = min(batch_size, 16)
        grid = utils.make_grid(generated_imgs[:num_images], nrow=4)
        plt.imshow(grid.permute(1, 2, 0))
        plt.show()
    else:
        # Display images in a single row
        fig, axs = plt.subplots(1, batch_size, figsize=(batch_size * 2, 2))
        for i in range(batch_size):
            axs[i].imshow(generated_imgs[i].permute(1, 2, 0))
            axs[i].axis('off')
        plt.show()